# 06 - Feature Engineering + Random Forest (Semi-Supervised)

**Objective**: Combine hand-crafted features with TF-IDF and train a Random Forest classifier.

**Key Innovation**: Semi-supervised learning using:
- **Gold Data**: CSV lookup tables (~2.5k per class after balancing)
- **Silver Data**: Pseudo-labeled unannotated LinkedIn data (~50 usable examples)

**Features**:
1. **TF-IDF**: Captures important job title keywords (1-2 grams)
2. **Manual Features**: Length, word count, keyword presence
3. **Career History**: Number of previous jobs, total positions (only in Silver/JSON)

**Data Usage**:
- **Training**: CSV (Gold) + Pseudo-labeled JSON (Silver)
- **In-Distribution Eval**: 80/20 split on CSV
- **Real-World Eval**: Annotated LinkedIn CVs

In [8]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append(os.path.abspath("../"))
from src.data.loader import load_label_lists, load_evaluation_dataset, balance_dataset

DATA_DIR = Path("../data")

In [9]:
# Load Gold Data (CSVs) - these have 'text' and 'label' columns
dept_gold, sen_gold = load_label_lists(DATA_DIR, fix_encoding=True, deduplicate=True, max_per_class=None)

# Load Silver Data (Pseudo-labeled Unannotated JSON)
silver_path = DATA_DIR / "processed/unannotated_pseudo_labeled.csv"
if silver_path.exists():
    silver_df = pd.read_csv(silver_path)
    
    # Extract department silver labels (rename 'title' to 'text' for consistency)
    dept_silver = silver_df[silver_df['dept_pseudo'].notna()][['title', 'dept_pseudo']].copy()
    dept_silver = dept_silver.rename(columns={'dept_pseudo': 'label', 'title': 'text'})
    
    # Extract seniority silver labels
    sen_silver = silver_df[silver_df['sen_pseudo'].notna()][['title', 'sen_pseudo']].copy()
    sen_silver = sen_silver.rename(columns={'sen_pseudo': 'label', 'title': 'text'})
    
    print(f"🥈 Loaded {len(dept_silver)} department silver labels")
    print(f"🥈 Loaded {len(sen_silver)} seniority silver labels")
else:
    dept_silver = pd.DataFrame(columns=['text', 'label'])
    sen_silver = pd.DataFrame(columns=['text', 'label'])
    print("⚠️  Silver data not found, falling back to Gold only.")

# Load Evaluation Data (has 'title' column)
eval_df = load_evaluation_dataset(DATA_DIR)

print(f"\n📊 Gold Department: {len(dept_gold):,} examples")
print(f"📊 Gold Seniority:  {len(sen_gold):,} examples")
print(f"📊 Real-world JSON CVs: {len(eval_df):,}")

Applying encoding fix...
Deduplicating department labels...
  Deduplication: 10145 -> 10145 (removed 0 duplicates)
Deduplicating seniority labels...
  Deduplication: 9428 -> 9428 (removed 0 duplicates)
🥈 Loaded 314 department silver labels
🥈 Loaded 314 seniority silver labels

📊 Gold Department: 10,145 examples
📊 Gold Seniority:  9,428 examples
📊 Real-world JSON CVs: 478


In [10]:
def extract_features(df):
    """
    Extract hand-crafted features from job titles and history.
    
    Works with DataFrames that have either 'text' or 'title' column.
    """
    # Auto-detect text column
    text_col = 'title' if 'title' in df.columns else 'text'
    
    # Get the text data as a clean 1D array
    texts = df[text_col].astype(str).values
    
    # Initialize features
    feats = {}
    
    # 1. Basic Title Stats
    feats['title_len'] = [len(t) for t in texts]
    feats['word_count'] = [len(t.split()) for t in texts]
    
    # 2. Career History (only available in Silver/JSON)
    if 'num_previous_jobs' in df.columns:
        feats['num_prev'] = df['num_previous_jobs'].fillna(0).values
        feats['total_pos'] = df['total_positions'].fillna(1).values
    else:
        feats['num_prev'] = [0] * len(texts)
        feats['total_pos'] = [1] * len(texts)
    
    # 3. Keyword Features
    sen_keywords = ['senior', 'lead', 'principal', 'manager', 'director', 'vp', 'head', 'junior', 'intern', 'trainee']
    for kw in sen_keywords:
        feats[f'has_{kw}'] = [int(kw in t.lower()) for t in texts]
    
    dept_keywords = {
        'it': ['engineer', 'developer', 'software', 'tech', 'data', 'it'],
        'sales': ['sales', 'account', 'revenue'],
        'mkt': ['marketing', 'brand', 'content'],
        'hr': ['hr', 'recruitment', 'people', 'talent'],
        'fin': ['finance', 'accounting', 'audit', 'tax']
    }
    for dept, keywords in dept_keywords.items():
        feats[f'has_{dept}'] = [int(any(k in t.lower() for k in keywords)) for t in texts]
    
    return pd.DataFrame(feats)

## 1. Department Classifier (Gold + Silver)

In [11]:
# A. Train/Test Split (In-Distribution on Gold)
gold_train, gold_test = train_test_split(
    dept_gold, test_size=0.2, random_state=42, stratify=dept_gold['label']
)

# B. Combine Gold Train + Silver
dept_train = pd.concat([gold_train, dept_silver], ignore_index=True)
print(f"Combined: {len(gold_train)} gold + {len(dept_silver)} silver = {len(dept_train)} total")

# C. Apply Tier 1 Data Balancing
dept_train, dept_weights = balance_dataset(
    dept_train, label_col='label', min_samples=500, max_samples=2000, return_weights=True
)
print(f"After balancing: {len(dept_train)} examples\n")

# D. Feature Extraction
X_train_manual = extract_features(dept_train)
X_test_manual = extract_features(gold_test)

# E. TF-IDF
tfidf = TfidfVectorizer(max_features=2000, ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(dept_train['text'])
X_test_tfidf = tfidf.transform(gold_test['text'])

# F. Combine Features
X_train = hstack([X_train_manual.values, X_train_tfidf])
X_test = hstack([X_test_manual.values, X_test_tfidf])
y_train = dept_train['label'].values
y_test = gold_test['label'].values

print(f"Training features: {X_train.shape}")
print(f"Testing features:  {X_test.shape}")

Combined: 8116 gold + 314 silver = 8430 total
Balancing: 8430 -> 9075 samples
  Class distribution: {'Marketing': 2000, 'Sales': 2000, 'Information Technology': 1054, 'Business Development': 521, 'Project Management': 500, 'Consulting': 500, 'Administrative': 500, 'Other': 500, 'Purchasing': 500, 'Human Resources': 500, 'Customer Support': 500}
After balancing: 9075 examples

Training features: (9075, 2019)
Testing features:  (2029, 2019)


In [12]:
# G. Train Random Forest
dept_rf = RandomForestClassifier(
    n_estimators=200, 
    max_depth=20,
    random_state=42, 
    class_weight='balanced',
    n_jobs=-1
)
dept_rf.fit(X_train, y_train)

# H. In-Distribution Evaluation (CSV test set)
y_pred_id = dept_rf.predict(X_test)
acc_id = accuracy_score(y_test, y_pred_id)

# I. Real-World Evaluation (Annotated JSON)
real_dept_df = eval_df[eval_df['department'].notna()].copy()
X_real_manual = extract_features(real_dept_df)
X_real_tfidf = tfidf.transform(real_dept_df['title'])  # eval_df has 'title' column
X_real = hstack([X_real_manual.values, X_real_tfidf])
y_real = real_dept_df['department'].values

y_pred_real = dept_rf.predict(X_real)
acc_real = accuracy_score(y_real, y_pred_real)

print("="*60)
print("DEPARTMENT CLASSIFICATION RESULTS")
print("="*60)
print(f"In-Distribution (CSV Test):  {acc_id:.4f}")
print(f"Real-World (Annotated JSON): {acc_real:.4f}")
print("="*60)

DEPARTMENT CLASSIFICATION RESULTS
In-Distribution (CSV Test):  0.9093
Real-World (Annotated JSON): 0.4079


## 2. Seniority Classifier (Gold + Silver)

In [13]:
# A. Train/Test Split (In-Distribution on Gold)
gold_train, gold_test = train_test_split(
    sen_gold, test_size=0.2, random_state=42, stratify=sen_gold['label']
)

# B. Combine Gold Train + Silver
sen_train = pd.concat([gold_train, sen_silver], ignore_index=True)
print(f"Combined: {len(gold_train)} gold + {len(sen_silver)} silver = {len(sen_train)} total")

# C. Apply Tier 1 Data Balancing
sen_train, sen_weights = balance_dataset(
    sen_train, label_col='label', min_samples=500, max_samples=2000, return_weights=True
)
print(f"After balancing: {len(sen_train)} examples\n")

# D. Feature Extraction
X_train_manual = extract_features(sen_train)
X_test_manual = extract_features(gold_test)

# E. TF-IDF
tfidf = TfidfVectorizer(max_features=2000, ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(sen_train['text'])
X_test_tfidf = tfidf.transform(gold_test['text'])

# F. Combine Features
X_train = hstack([X_train_manual.values, X_train_tfidf])
X_test = hstack([X_test_manual.values, X_test_tfidf])
y_train = sen_train['label'].values
y_test = gold_test['label'].values

print(f"Training features: {X_train.shape}")
print(f"Testing features:  {X_test.shape}")

Combined: 7542 gold + 314 silver = 7856 total
Balancing: 7856 -> 6012 samples
  Class distribution: {'Senior': 2000, 'Lead': 2000, 'Director': 818, 'Management': 694, 'Junior': 500}
After balancing: 6012 examples

Training features: (6012, 2019)
Testing features:  (1886, 2019)


In [14]:
# G. Train Random Forest
sen_rf = RandomForestClassifier(
    n_estimators=200, 
    max_depth=20,
    random_state=42, 
    class_weight='balanced',
    n_jobs=-1
)
sen_rf.fit(X_train, y_train)

# H. In-Distribution Evaluation (CSV test set)
y_pred_id = sen_rf.predict(X_test)
acc_id = accuracy_score(y_test, y_pred_id)

# I. Real-World Evaluation (Annotated JSON)
real_sen_df = eval_df[eval_df['seniority'].notna()].copy()
X_real_manual = extract_features(real_sen_df)
X_real_tfidf = tfidf.transform(real_sen_df['title'])
X_real = hstack([X_real_manual.values, X_real_tfidf])
y_real = real_sen_df['seniority'].values

y_pred_real = sen_rf.predict(X_real)
acc_real = accuracy_score(y_real, y_pred_real)

print("="*60)
print("SENIORITY CLASSIFICATION RESULTS")
print("="*60)
print(f"In-Distribution (CSV Test):  {acc_id:.4f}")
print(f"Real-World (Annotated JSON): {acc_real:.4f}")
print("="*60)

SENIORITY CLASSIFICATION RESULTS
In-Distribution (CSV Test):  0.9380
Real-World (Annotated JSON): 0.4310
